<a href="https://colab.research.google.com/github/prasanthkavin/BizCardX-Extracting-Business-Card-Data-with-OCR/blob/main/xyma_analytics_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from scipy.integrate import trapz, simps
from scipy.optimize import curve_fit
from scipy.fft import fft, fftfreq

# Task 1: Visualize the ultrasound signal with peak and valley tracking
df = pd.read_csv('1 (17).csv')
amplitude = df['average(-A)']
time = df['Time']

peaks = argrelextrema(amplitude.values, comparator_func=max, order=10)[0]
valleys = argrelextrema(amplitude.values, comparator_func=min, order=10)[0]

selected_peaks_valleys = pd.DataFrame(columns=['Amplitude', 'Time'])

for index in peaks:
    selected_peaks_valleys = selected_peaks_valleys.append({'Amplitude': amplitude[index], 'Time': time[index]}, ignore_index=True)

for index in valleys:
    selected_peaks_valleys = selected_peaks_valleys.append({'Amplitude': amplitude[index], 'Time': time[index]}, ignore_index=True)

selected_peaks_valleys.to_excel('peaks_valleys.xlsx', index=False)

plt.plot(time, amplitude)
plt.scatter(time[peaks], amplitude[peaks], color='r', label='Peaks')
plt.scatter(time[valleys], amplitude[valleys], color='g', label='Valleys')
plt.xlabel('Time ')
plt.ylabel('Amplitude')
plt.title('Ultrasound Signal with Peaks and Valleys')
plt.legend()
plt.show()

# Task 2: Calculate the area under the curve using numerical integration methods
area_trapezoidal = trapz(amplitude, time)
area_midpoint = np.sum((time[1:] - time[:-1]) * amplitude[:-1])
area_simpsons = simps(amplitude, time)

print('Area under the curve (Trapezoidal Rule):', area_trapezoidal)
print('Area under the curve (Midpoint Rule):', area_midpoint)
print('Area under the curve (Simpson\'s Rule):', area_simpsons)

# Task 3: Fit a Gaussian curve to the first peak
def gaussian(x, a, b, c):
    return a * np.exp(-(x - b)**2 / (2 * c**2))

first_peak_amplitude = amplitude[peaks[0]]
first_peak_time = time[peaks[0]]

popt, pcov = curve_fit(gaussian, time, amplitude, p0=[first_peak_amplitude, first_peak_time, 1])

a_fit, b_fit, c_fit = popt
gaussian_equation = f'Amplitude * exp(-(x - Time)**2 / (2 * Sigma**2)), where Amplitude = {a_fit}, Time = {b_fit}, Sigma = {c_fit}'

plt.plot(time, amplitude)
plt.plot(time, gaussian(time, a_fit, b_fit, c_fit), 'r--', label='Gaussian Fit')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Gaussian Fit to First Peak')
plt.legend()
plt.show()

# Task 4: Compute the FFT and plot the frequency spectrum
fft_values = fft(amplitude)
freq = fftfreq(len(amplitude), d=(time[1] - time[0]))

plt.plot(freq, np.abs(fft_values))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.title('Frequency Spectrum')
plt.show()
